In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, latest_weights_file_path
from train import get_model, get_ds, run_validation


C:\Users\ADMIN\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

## Load the pretrained weights
# model_filename = latest_weights_file_path(config)
# state = torch.load(model_filename)
# model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 798
Max length of target sentence: 850


In [ ]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

In [ ]:
from translate import translate

In [ ]:
t = translate("Why do I need to translate this?")

In [ ]:
t = translate(34)